In [3]:
from datasets import load_dataset

ds = load_dataset("allenai/soda", split="train")

d:\SadRobot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import pandas as pd
import json

In [13]:
subset = ds.shuffle(seed=42).select(range(100000))

In [2]:
subset = ds

NameError: name 'ds' is not defined

In [19]:
counseling = subset.filter(lambda ex: "counseling" in ex["narrative"].lower())

Filter: 100%|██████████| 1191582/1191582 [01:49<00:00, 10861.81 examples/s]


In [20]:
counseling_df = counseling.to_pandas()

In [21]:
counseling_df.head()

,head,relation,tail,literal,narrative,dialogue,speakers,PersonX,PersonY,PersonZ,original_index,split,head_answer,pmi_head_answer,relation_tail_answer,pmi_relation_tail_answer
0,PersonX finds hope and self-esteem in counseling,xReact,optimistic,Shan finds hope and self-esteem in counseling....,Shan seeks counseling to help address his low ...,"[I just feel like I'm not good enough, you kno...","[Shan, Counselor, Shan, Counselor, Shan, Couns...",Shan,,,6028012,train,Yes,Yes,Yes,No
1,PersonX begins a counseling session,xEffect,is able to control his anger,Zuleika begins a counseling session. Now Zulei...,Zuleika begins a counseling session. The couns...,"[Counselor: So, what's been going on that's be...","[Zuleika, Counselor, Zuleika, Counselor, Zulei...",Zuleika,,,6062358,train,Yes,Yes,Yes,Yes
2,PersonX suggests seeking counseling,xEffect,receives counseling,Chaz suggests seeking counseling. Now Chaz rec...,Chaz had been dealing with a lot of stress fro...,[I feel like I'm finally able to talk about my...,"[Chaz, Husband, Chaz, Husband, Chaz, Husband]",Chaz,,,6073460,train,Yes,Yes,Yes,Yes
3,PersonX begins a counseling session,xReact,safe,Ahron begins a counseling session. Now Ahron f...,Ahron begins a counseling session in which the...,"[I'm not sure where to start., That's okay. We...","[Ahron, Counselor, Ahron, Counselor, Ahron, Co...",Ahron,,,6099833,train,Yes,Yes,Unknown,No
4,PersonX goes away to school,xNeed,to apply,Jeb applied. Jeb goes away to school.,Jeb went to the college counseling office and ...,"[So I've been thinking about what you said, an...","[Jeb, College Counselor, Jeb, College Counselo...",Jeb,,,6189113,train,Yes,Yes,Unknown,Yes


In [22]:
counseling_df.to_csv("soda_counseling.csv")

In [3]:
feel_better_df = pd.read_csv("soda_feel_better.csv")

In [4]:
df_filtered = feel_better_df

In [6]:
df_filtered.head()

,Unnamed: 0,head,relation,tail,literal,narrative,dialogue,speakers,PersonX,PersonY,PersonZ,original_index,split,head_answer,pmi_head_answer,relation_tail_answer,pmi_relation_tail_answer
0,1,PersonX talks to PersonY in the afternoon,xIntent,to make PersonY feel better,Lilibeth talks to Katelyn in the afternoon bec...,Katelyn is sad because her best friend moved a...,"[""Hey, it's Lilibeth. How are you holding up?""...",['Lilibeth' 'Katelyn' 'Lilibeth' 'Katelyn' 'Li...,Lilibeth,Katelyn,NaN,6032411,train,Yes,Yes,Yes,Yes
1,6,PersonX mentions some event to PersonY,xIntent,to make PersonY feel better,Seleste mentions some event to Mitzy because S...,"He had seen her looking sad, and he thought th...","['Hi, Mitzy. You seem a little down today. Is ...",['Seleste' 'Mitzy' 'Seleste' 'Mitzy' 'Seleste'...,Seleste,Mitzy,NaN,6045902,train,Unknown,No,Yes,Yes
2,7,PersonX relates to PersonY,xIntent,to make PersonY feel better,Gavin relates to Gianella because Gavin wants ...,Gavin hugs Gianella because she wants him to f...,"['Hey, are you okay? You look like you could u...",['Gavin' 'Gianella' 'Gavin' 'Gianella' 'Gavin'...,Gavin,Gianella,NaN,6046160,train,Yes,Yes,Yes,Yes
3,8,PersonX strokes PersonY's hair,xIntent,to make PersonY feel better,Nick strokes Stori's hair because Nick wants t...,Nick is stroking Stori's hair because she want...,"[""Hey, it's okay. I'm here for you.""\n 'I know...",['Nick' 'Stori' 'Nick' 'Stori' 'Nick' 'Stori' ...,Nick,Stori,NaN,6055258,train,Yes,Yes,Yes,Yes
4,9,PersonX offers a drink to PersonY,xIntent,to make PersonY feel better,Caelan offers a drink to Bethany because Caela...,Caelan offered Bethany a drink of water becaus...,"[""Hi, Bethany. I'm so sorry to hear about your...",['Caelan' 'Bethany' 'Caelan' 'Bethany' 'Caelan...,Caelan,Bethany,NaN,6061097,train,Yes,Yes,Yes,Yes


In [7]:
# Let's examine the actual data format first
sample_row = df_filtered.iloc[6]
print("Raw dialogue:", repr(sample_row["dialogue"]))
print("Raw speakers:", repr(sample_row["speakers"]))


def parse_speakers(s: str) -> List[str]:
    """Parse speakers from numpy array format like "['Shan' 'Counselor' 'Shan' 'Counselor']" """
    if s is None:
        return []
    s = s.strip()
    
    if s.startswith("['") and s.endswith("']"):
        inner = s[2:-2]  # Remove "['" and "']"
        # Replace newlines with spaces to normalize the format
        inner = inner.replace('\n', '')
        parts = inner.split("' '")
        # Clean up each part by removing quotes and whitespace
        result = []
        for part in parts:
            part = part.strip()
            if not part:
                continue
            # Remove any remaining quotes
            # if part.startswith("'") and part.endswith("'"):
            #     part = part[1:-1]
            # elif part.startswith('"') and part.endswith('"'):
            #     part = part[1:-1]
            if part.strip():
                result.append(part.strip())
        # clean = [x.strip(" '") for x in result if "  " not in x]
        return result
    
    # Fallback to general parsing
    try:
        val = ast.literal_eval(s)
        if isinstance(val, (list, tuple)):
            return [str(x) for x in val]
    except Exception:
        pass
    
    return [s] if s else []


def parse_dialogue(s: str) -> List[str]:
    """Parse dialogue from JSON string format like '["text1"\\n \\'text2\\'\\n "text3"]' """
    if s is None:
        return []
    s = s.strip()
    
    # Handle dialogue format like '["text1"\\n \\'text2\\'\\n "text3"]'
    if s.startswith('["') and s.endswith('"]'):
        # Remove outer quotes and brackets
        inner = s[2:-2]  # Remove '["' and '"]'
        # Split by actual newline character, not escaped
        parts = inner.split('\n')
        result = []
        for part in parts:
            part = part.strip()
            if not part:
                continue
            # Remove leading/trailing quotes and spaces
            if part.startswith('"') and part.endswith('"'):
                part = part[1:-1]
            elif part.startswith("'") and part.endswith("'"):
                part = part[1:-1]
            # Unescape quotes
            part = part.replace("\\'", "'").replace('\\"', '"')
            if part.strip():
                result.append(part.strip())
        return result
    
    # Fallback to general parsing
    try:
        val = ast.literal_eval(s)
        if isinstance(val, (list, tuple)):
            return [str(x) for x in val]
    except Exception:
        pass
    
    return [s] if s else []

def build_messages(dialogue: List[str], speakers: List[str]) -> Optional[List[Dict[str, str]]]:
    """
    Build OpenAI chat messages with roles:
      - first speaker -> role "user"
      - other speakers -> role "assistant"
    Uses only the dialogue text and speaker order.

    Returns None if invalid/misaligned.
    """
    if not dialogue or not speakers or len(dialogue) != len(speakers):
        print("length mismatch:", len(dialogue), len(speakers))
        return None

    # Get the first speaker to determine who is the user
    first_speaker = speakers[0] if speakers else None
    if not first_speaker:
        return None

    messages: List[Dict[str, str]] = []
    for sp, text in zip(speakers, dialogue):
        if not isinstance(text, str):
            continue
        # First speaker is always "user", others are "assistant"
        role = "user" if sp == first_speaker else "assistant"
        # Normalize whitespace a bit
        content = re.sub(r"\s+\n", "\n", text.strip())
        if content:
            messages.append({"role": role, "content": content})

    # Collapse any consecutive messages of the same role (defensive)
    collapsed: List[Dict[str, str]] = []
    for m in messages:
        if collapsed and collapsed[-1]["role"] == m["role"]:
            collapsed[-1]["content"] = (collapsed[-1]["content"] + "\n" + m["content"]).strip()
        else:
            collapsed.append(m)

    # Ensure we have both user and assistant messages
    roles_present = {m["role"] for m in collapsed}
    if not {"user", "assistant"}.issubset(roles_present):
        return None

    return collapsed


def dataframe_to_examples(df: pd.DataFrame, dialogue_col: str, speakers_col: str) -> List[Dict]:
    """
    Convert a DataFrame into a list of {"messages": [...]} examples.
    Keeps only rows that contain a counselor/therapist in `speakers`.
    """
    results = []
    print("total rows:", len(df))
    for _, row in df.iterrows():
        dialogue_list = parse_dialogue(row.get(dialogue_col))
        speakers_list = parse_speakers(row.get(speakers_col))

        messages = build_messages(dialogue_list, speakers_list)
        if messages:
            results.append({"messages": messages})

    return results



Raw dialogue: '[\'Hi, are you okay? You look a little down.\'\n "Yeah, I\'m just feeling a little down today."\n "Oh, I\'m sorry to hear that. Is there anything I can do to help make you feel better?"\n \'Well, maybe getting me a drink or something to eat would help.\'\n "Okay, sure. I\'ll be right back." \'Here you go. I hope this helps.\'\n \'Thank you. It does help. I appreciate it.\']'
Raw speakers: "['Garth' 'Jamar' 'Garth' 'Jamar' 'Garth' 'Garth' 'Jamar']"


NameError: name 'List' is not defined

In [8]:
import argparse
import ast
import json
import re
from typing import List, Dict, Tuple, Optional

import pandas as pd


def parse_speakers(s: str) -> List[str]:
    """Parse speakers from numpy array format like "['Shan' 'Counselor' 'Shan' 'Counselor']" """
    if s is None:
        return []
    s = s.strip()
    
    if s.startswith("['") and s.endswith("']"):
        inner = s[2:-2]  # Remove "['" and "']"
        # Replace newlines with spaces to normalize the format
        inner = inner.replace('\n', '')
        parts = inner.split("' '")
        # Clean up each part by removing quotes and whitespace
        result = []
        for part in parts:
            part = part.strip()
            if not part:
                continue
            if part.strip():
                result.append(part.strip())
        return result
    
    # Fallback to general parsing
    try:
        val = ast.literal_eval(s)
        if isinstance(val, (list, tuple)):
            return [str(x) for x in val]
    except Exception:
        pass
    
    return [s] if s else []


def parse_dialogue(s: str) -> List[str]:
    """Parse dialogue from the actual format - WORKING VERSION"""
    if s is None:
        return []
    s = s.strip()
    
    # Try to parse as Python list first
    try:
        val = ast.literal_eval(s)
        if isinstance(val, list):
            # If it's a list with one concatenated string, split it
            if len(val) == 1 and isinstance(val[0], str):
                # Split the concatenated dialogue by sentence boundaries
                text = val[0]
                
                # Split by punctuation followed by capital letters, keeping the punctuation
                parts = re.split(r'([.!?])(?=[A-Z])', text)
                
                # Combine text with punctuation
                result = []
                for i in range(0, len(parts), 2):
                    if i < len(parts):
                        sentence = parts[i]
                        if i + 1 < len(parts):
                            sentence += parts[i + 1]  # Add the punctuation
                        sentence = sentence.strip()
                        if sentence:
                            # Unescape quotes
                            sentence = sentence.replace("\\'", "'").replace('\\"', '"')
                            result.append(sentence)
                
                # Handle the last part if it doesn't end with punctuation + capital
                if len(parts) % 2 == 1:  # Odd number means there's a leftover part
                    last_part = parts[-1].strip()
                    if last_part and last_part not in [r.strip() for r in result]:  # Avoid duplicates
                        # Unescape quotes
                        last_part = last_part.replace("\\'", "'").replace('\\"', '"')
                        result.append(last_part)
                
                return result
            else:
                # It's already a proper list
                return [str(x) for x in val]
    except Exception:
        pass
    
    return [s] if s else []


def build_messages(dialogue: List[str], speakers: List[str]) -> Optional[List[Dict[str, str]]]:
    """
    Build OpenAI chat messages with roles:
      - first speaker -> role "user"
      - other speakers -> role "assistant"
    Uses only the dialogue text and speaker order.

    Returns None if invalid/misaligned.
    """
    if not dialogue or not speakers or len(dialogue) != len(speakers):
        print("length mismatch:", len(dialogue), len(speakers))
        return None

    # Get the first speaker to determine who is the user
    first_speaker = speakers[0] if speakers else None
    if not first_speaker:
        return None

    messages: List[Dict[str, str]] = []
    for sp, text in zip(speakers, dialogue):
        if not isinstance(text, str):
            continue
        # First speaker is always "user", others are "assistant"
        role = "user" if sp == first_speaker else "assistant"
        # Normalize whitespace a bit
        content = re.sub(r"\s+\n", "\n", text.strip())
        if content:
            messages.append({"role": role, "content": content})

    # Collapse any consecutive messages of the same role (defensive)
    collapsed: List[Dict[str, str]] = []
    for m in messages:
        if collapsed and collapsed[-1]["role"] == m["role"]:
            collapsed[-1]["content"] = (collapsed[-1]["content"] + "\n" + m["content"]).strip()
        else:
            collapsed.append(m)

    # Ensure we have both user and assistant messages
    roles_present = {m["role"] for m in collapsed}
    if not {"user", "assistant"}.issubset(roles_present):
        return None

    return collapsed


def dataframe_to_examples(df: pd.DataFrame, dialogue_col: str, speakers_col: str) -> List[Dict]:
    """
    Convert a DataFrame into a list of {"messages": [...]} examples.
    """
    results = []
    print("total rows:", len(df))
    for _, row in df.iterrows():
        dialogue_list = parse_dialogue(row.get(dialogue_col))
        speakers_list = parse_speakers(row.get(speakers_col))

        messages = build_messages(dialogue_list, speakers_list)
        if messages:
            results.append({"messages": messages})

    return results


In [9]:
# Test the WORKING version
sample_row = df_filtered.iloc[6]
test_speakers = parse_speakers(sample_row["speakers"])
test_dialogue = parse_dialogue(sample_row["dialogue"])

print("Test speakers:", test_speakers)
print("Test dialogue:", test_dialogue)
print("Lengths:", len(test_speakers), len(test_dialogue))

# Test the build_messages function
test_messages = build_messages(test_dialogue, test_speakers)
if test_messages:
    print("\nGenerated messages:")
    for i, msg in enumerate(test_messages):
        print(f"{i+1}. {msg['role']}: {msg['content'][:100]}...")
else:
    print("No messages generated")


Test speakers: ['Garth', 'Jamar', 'Garth', 'Jamar', 'Garth', 'Garth', 'Jamar']
Test dialogue: ['Hi, are you okay? You look a little down.', "Yeah, I'm just feeling a little down today.", "Oh, I'm sorry to hear that. Is there anything I can do to help make you feel better?", 'Well, maybe getting me a drink or something to eat would help.', "Okay, sure. I'll be right back.", 'Here you go. I hope this helps.', 'Thank you. It does help. I appreciate it.']
Lengths: 7 7

Generated messages:
1. user: Hi, are you okay? You look a little down....
2. assistant: Yeah, I'm just feeling a little down today....
3. user: Oh, I'm sorry to hear that. Is there anything I can do to help make you feel better?...
4. assistant: Well, maybe getting me a drink or something to eat would help....
5. user: Okay, sure. I'll be right back.
Here you go. I hope this helps....
6. assistant: Thank you. It does help. I appreciate it....


In [10]:
# Test with the full dataset
examples = dataframe_to_examples(df_filtered, "dialogue", "speakers")
print(f"Generated {len(examples)} examples")

# Show a sample
if examples:
    print("\nSample example:")
    sample = examples[0]
    for i, msg in enumerate(sample["messages"]):
        print(f"{i+1}. {msg['role']}: {msg['content'][:150]}...")
        print()


total rows: 647
length mismatch: 9 8
length mismatch: 15 14
length mismatch: 10 11
length mismatch: 11 9
length mismatch: 11 12
length mismatch: 10 9
length mismatch: 14 15
length mismatch: 7 8
length mismatch: 5 7
length mismatch: 14 13
length mismatch: 5 6
length mismatch: 7 6
length mismatch: 6 5
length mismatch: 7 8
length mismatch: 1 7
length mismatch: 9 8
length mismatch: 15 13
length mismatch: 5 6
length mismatch: 14 15
length mismatch: 12 13
length mismatch: 8 9
length mismatch: 12 13
length mismatch: 8 9
length mismatch: 6 7
length mismatch: 8 9
length mismatch: 12 11
length mismatch: 11 10
length mismatch: 8 7
length mismatch: 10 9
length mismatch: 8 7
length mismatch: 13 11
length mismatch: 7 6
length mismatch: 12 13
length mismatch: 6 7
length mismatch: 15 11
length mismatch: 14 13
length mismatch: 10 13
length mismatch: 12 11
length mismatch: 13 11
length mismatch: 5 6
length mismatch: 6 7
length mismatch: 6 7
length mismatch: 6 7
length mismatch: 11 9
length mismatch: 10 

In [26]:
# Now let's test with the full dataset
examples = dataframe_to_examples(df_filtered, "dialogue", "speakers")
print(f"Generated {len(examples)} examples")

total rows: 647
length mismatch: 9 8
length mismatch: 15 14
length mismatch: 10 11
length mismatch: 11 9
length mismatch: 11 12
length mismatch: 10 9
length mismatch: 14 15
length mismatch: 7 8
length mismatch: 5 7
length mismatch: 14 13
length mismatch: 5 6
length mismatch: 7 6
length mismatch: 6 5
length mismatch: 7 8
length mismatch: 1 7
length mismatch: 9 8
length mismatch: 15 13
length mismatch: 5 6
length mismatch: 14 15
length mismatch: 12 13
length mismatch: 8 9
length mismatch: 12 13
length mismatch: 8 9
length mismatch: 6 7
length mismatch: 8 9
length mismatch: 12 11
length mismatch: 11 10
length mismatch: 8 7
length mismatch: 10 9
length mismatch: 8 7
length mismatch: 13 11
length mismatch: 7 6
length mismatch: 12 13
length mismatch: 6 7
length mismatch: 15 11
length mismatch: 14 13
length mismatch: 10 13
length mismatch: 12 11
length mismatch: 13 11
length mismatch: 5 6
length mismatch: 6 7
length mismatch: 6 7
length mismatch: 6 7
length mismatch: 11 9
length mismatch: 10 

In [11]:
# Show a sample of the generated examples
if examples:
    print("Sample example:")
    sample = examples[0]
    for i, msg in enumerate(sample["messages"]):
        print(f"{i+1}. {msg['role']}: {msg['content'][:150]}...")
        print()
else:
    print("No examples generated")

Sample example:
1. user: Hey, it's Lilibeth. How are you holding up?...

2. assistant: Not great. I miss my best friend so much....

3. user: I'm so sorry. What was your favorite memory with her?...

4. assistant: We used to spend every Saturday together. We would go to the park and then get ice cream. Just talking and laughing together was the best part....

5. user: That sounds really nice. I'm sure you'll make new memories with other friends, but it's totally natural to miss her right now....

6. assistant: Yeah, I guess you're right. Thanks for talking to me, Lilibeth....

7. user: Of course! Anytime you need to talk, I'm here for you....



In [12]:
with open("feel_better_2.jsonl", "w", encoding="utf-8") as f:
    for ex in examples[70:140]:
        json.dump(ex, f, ensure_ascii=False)
        f.write("\n")

print("✅ Saved to feel_better_2.jsonl")

✅ Saved to feel_better_2.jsonl


In [14]:
input_path = "feel_better_2.jsonl"
output_path = "feel_better_2_cleaned.jsonl"

with open(input_path, "r", encoding="utf-8") as infile, open(output_path, "w", encoding="utf-8") as outfile:
    for line in infile:
        data = json.loads(line)
        if "messages" in data and len(data["messages"]) > 0:
            # Check if the last message is from the user
            if data["messages"][-1]["role"] == "user":
                data["messages"].pop()  # remove last user message
        # Only write conversations that still have messages
        if data.get("messages"):
            json.dump(data, outfile, ensure_ascii=False)
            outfile.write("\n")

print("✅ Cleaned file saved as finetune_soda_50_cleaned.jsonl")

✅ Cleaned file saved as finetune_soda_50_cleaned.jsonl
